In [93]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk import LancasterStemmer
import joblib

In [125]:
if not nltk.download('stopwords'):
    nltk.download('stopwords')

#print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jatin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [95]:
data = pd.read_csv('./data/train.csv')
data.shape
data.dtypes
data.isna().sum()
data = data.fillna('')
data['article'] = data['title'] + data['author']
data.drop(columns=['id'], inplace=True)

(20800, 5)

id         int64
title     object
author    object
text      object
label      int64
dtype: object

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [101]:
data['article'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - BreitbartDaniel J. Flynn'

In [102]:
# X = data.drop(columns='label', axis=1)
# Y = data['label']

In [103]:
porterstemmer = PorterStemmer()
lancasterstemmer = LancasterStemmer()

joblib.dump(porterstemmer, './models/porterstemmer.joblib', compress=0, protocol=None, cache_size=None)

joblib.dump(lancasterstemmer, './models/lancasterstemmer.joblib', compress=0, protocol=None, cache_size=None)




['./models/lancasterstemmer.joblib']

In [104]:
ps = joblib.load('./models/porterstemmer.joblib')
ls = joblib.load('./models/lancasterstemmer.joblib')

In [105]:
# stemmed_word = re.sub('[`~!@#$%^&*()\\]\\[+={}/|:;\"\'<>,.?-_]', X)
def perform_stemming(content):
    remove_special_char = re.sub('[^a-zA-Z]',' ', content)
    string_to_lower = remove_special_char.lower().split()
    #string_split = string_to_lower.split()
    remove_word = [word for word in string_to_lower if not word in stopwords.words('english')]
    stem = [ps.stem(word) for word in remove_word]
    return stem

In [106]:
data['article'] = data['article'].apply(perform_stemming)

In [107]:
#data = perform_stemming_test(data,['article'])

In [108]:
data['article'][0]

['hous',
 'dem',
 'aid',
 'even',
 'see',
 'comey',
 'letter',
 'jason',
 'chaffetz',
 'tweet',
 'itdarrel',
 'lucu']

In [109]:
# def vectorize_data(data):
#     vectorizer = CountVectorizer()
#     bag_of_words = CountVectorizer(max_features=5000, ngram_range=(1, 5))
#     bag_of_words.fit(data)
#     return features

In [110]:
X = data['article'].map(' '.join)
y = data['label']

In [111]:
from sklearn.model_selection import train_test_split

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=0)
print(y_train)

8118     0
16465    1
16197    0
3808     0
8544     0
        ..
13123    1
19648    1
9845     0
10799    0
2732     0
Name: label, Length: 4160, dtype: int64


In [113]:
X_train.shape

(4160,)

In [114]:
y_test.shape

(16640,)

In [115]:
def vectorize_data(vectorizer, X_train, X_test):
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    return X_train, X_test

In [116]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

X_test

<16640x9612 sparse matrix of type '<class 'numpy.int64'>'
	with 135806 stored elements in Compressed Sparse Row format>

In [117]:
print(X_test)

  (0, 872)	1
  (0, 2275)	1
  (0, 5533)	1
  (0, 5589)	1
  (0, 5956)	1
  (0, 6212)	1
  (0, 6396)	1
  (0, 8455)	1
  (0, 9565)	1
  (1, 2275)	1
  (1, 2486)	1
  (1, 4491)	1
  (1, 5533)	1
  (1, 5649)	1
  (1, 5846)	1
  (1, 7678)	1
  (1, 9287)	1
  (2, 519)	1
  (2, 7630)	1
  (2, 7875)	1
  (3, 527)	1
  (3, 1417)	1
  (3, 2879)	1
  (3, 2995)	1
  (3, 3516)	1
  :	:
  (16637, 2155)	1
  (16637, 2746)	1
  (16637, 3180)	1
  (16637, 3320)	1
  (16637, 3639)	1
  (16637, 4893)	1
  (16637, 5533)	1
  (16637, 7701)	1
  (16637, 8490)	1
  (16637, 9565)	1
  (16638, 768)	1
  (16638, 4448)	1
  (16638, 5193)	1
  (16639, 1503)	1
  (16639, 2163)	1
  (16639, 3129)	1
  (16639, 5536)	1
  (16639, 5740)	1
  (16639, 6015)	1
  (16639, 6183)	1
  (16639, 6359)	1
  (16639, 7452)	1
  (16639, 8001)	1
  (16639, 8879)	1
  (16639, 9382)	1


In [118]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer()
# X = vectorizer.fit_transform(corpus)

In [119]:
X_train.shape

(4160, 9612)

In [120]:
X_test.shape

(16640, 9612)

In [121]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
reg = LR.fit(X_train, y_train)
reg_pred = reg.predict(X_test)
reg_pred

array([0, 1, 1, ..., 0, 1, 1], dtype=int64)

In [122]:
from sklearn import metrics

print("Logistic Regression model accuracy(in %):", 
metrics.accuracy_score(y_test, reg_pred)*100)

Logistic Regression model accuracy(in %): 95.90745192307692


In [123]:
from sklearn.neighbors import KNeighborsClassifier
 
knn = KNeighborsClassifier(n_neighbors = 2)
 
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_test)
 
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, knn_pred))
 
print(classification_report(y_test, knn_pred))

[[2462 5837]
 [  33 8308]]
              precision    recall  f1-score   support

           0       0.99      0.30      0.46      8299
           1       0.59      1.00      0.74      8341

    accuracy                           0.65     16640
   macro avg       0.79      0.65      0.60     16640
weighted avg       0.79      0.65      0.60     16640



In [124]:
error_rate = []
 
for i in range(1, 40):
     
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_train, y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))
 
plt.figure(figsize =(10, 6))
plt.plot(range(1, 40), error_rate, color ='blue',
                linestyle ='dashed', marker ='o',
         markerfacecolor ='red', markersize = 10)
 
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

NameError: name 'plt' is not defined

In [ ]:
from sklearn.ensemble import RandomForestRegressor
 
rf = RandomForestRegressor(n_estimators = 100, random_state = 0)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
print("Random Forest model accuracy(in %):", 
metrics.accuracy_score(y_test, rf_pred)*100)

In [ ]:
X_grid = np.arrange(min(X_train), max(X_train), 0.01)
 
# reshape for reshaping the data into a len(X_grid)*1 array,
# i.e. to make a column out of the X_grid value                 
X_grid = X_grid.reshape((len(X_grid), 1))
 
# Scatter plot for original data
plt.scatter(X_train, y_train, color = 'blue') 
 
# plot predicted data
plt.plot(X_grid, rf.predict(X_grid),
         color = 'green')
plt.title('Random Forest Regression')
plt.xlabel('Position level')
plt.ylabel('Salary')
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor


In [ ]:
def model(model,X_train, y_train, y_test):
    mod = model.fit(X_train, y_train)
    y_pred = mod.predict(X_test)
    #print("model accuracy(in %):", metrics.accuracy_score(y_test, y_pred)*100)
    print(classification_report(y_test, y_pred))

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 10)
model(knn, X_train, y_train, y_test)

              precision    recall  f1-score   support

           0       0.99      0.06      0.12      8299
           1       0.52      1.00      0.68      8341

    accuracy                           0.53     16640
   macro avg       0.75      0.53      0.40     16640
weighted avg       0.75      0.53      0.40     16640



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
 
knn = KNeighborsClassifier(n_neighbors = 2)
model(knn)
print(classification_report(y_test, knn_pred))

 K-nearest  neighbor (KNN),  support  vector machine  (SVM),  logistic  regression  (LR),  linear  support  vector machine (LSVM), decision tree (DT), 